In [1]:
from fastfusion import arch
from fastfusion import constraints
from fastfusion import workload

variables = {
    "global_cycle_period": 1e-9,
    "tech_node": 1e-9
}

main_memory = arch.Memory(
    name="MainMemory",
    component_class="DRAM",
    attributes=dict(
        _size=4*1024*1024*1024*8,
        _bandwidth_reads_plus_writes_per_cycle=614e9 / 8192 * variables["global_cycle_period"],
        _datawidth=8,
        width=8192,
        depth=4*1024*1024*1024*8 / 8192,
    ))
main_memory.constraints.tensors.keep = "~Intermediates()"

global_buffer = arch.Memory(
    name="GlobalBuffer",
    component_class="SRAM",
    attributes=dict(
        _size=1024*1024*128*8,
        _datawidth=8,
        _bandwidth_reads_plus_writes_per_cycle=1,
        width=16384,
        depth=1024*1024*128*8 / 16384,
    )
)
global_buffer.constraints.tensors.keep = "All()"
global_buffer.constraints.dataflow.tensor_order_options = [["MainMemory.tensors()", "~MainMemory.tensors()"]]

local_buffer = arch.Memory(
    name="LocalBuffer",
    component_class="SRAM",
    attributes=dict(
        _size=1024*1024*4*8,
        _datawidth=8,
        _bandwidth_reads_plus_writes_per_cycle=1,
        width=16384,
        depth=1024*1024*4*8 / 16384,
    ),
    spatial=[{"name": "Z", "fanout": 4}],
)
local_buffer.constraints.tensors.keep = "input | output"
local_buffer.constraints.dataflow.tensor_order_options = [["output", "input"]]
local_buffer.constraints.spatial.append(constraints.Spatial(
    name="Z",
    min_utilization="1 if len(All()) > 2 else 0",
    loop_bounds=[{"expression": "weight.rank_variables()", "operator": "==", "value": 1}]
))

register = arch.Memory(
    name="Register",
    component_class="dummy_storage",
    spatial=[{"name": "X", "fanout": 128}, {"name": "Y", "fanout": 128}],
    attributes=dict(
        _size=8,
        _datawidth=8,
    )
)
register.constraints.tensors.keep = "weight"
register.constraints.tensors.bypass = "~weight"
register.constraints.spatial.append(constraints.Spatial(
    name="X",
    min_utilization=1,
    loop_bounds=[{"expression": "input.rank_variables()", "operator": "==", "value": 1}]
))
register.constraints.spatial.append(constraints.Spatial(
    name="Y",
    min_utilization=1,
    loop_bounds=[{"expression": "output.rank_variables()", "operator": "==", "value": 1}]
))

compute = arch.Compute(
    name="MAC",
    component_class="intmac",
    attributes=dict(
        multiplier_width=8,
        adder_width=16,
    )
)
compute.constraints.misc.enabled = "len(All()) == 3"

scalar_unit = arch.Compute(
    name="scalar_unit",
    component_class="dummy_compute",
    attributes=dict(
        area=1,
        _computes_per_cycle=128
    )
)
scalar_unit.constraints.misc.enabled = "len(All()) == 2"


architecture = arch.Arch(
    nodes=[
        main_memory,
        global_buffer,
        local_buffer,
        scalar_unit,
        register,
        compute,
    ]
)

# workload = workload.Workload(
#     einsums=[
#         workload.Einsum(
#             name="Einsum 1",
            
#         )
#     ]
# )
    